In [1]:
#European Option
import math
import numpy as np
import numpy.random as npr
from pylab import plt,mpl


d:\PROGRAMMING\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\PROGRAMMING\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
d:\PROGRAMMING\anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def gen_sn(M,I,anti_paths=True,mo_match = True):
    '''Function togenerate random number for simulation.
    
    Parameters
    M: number of intervals for discretization
    I: int number of paths to be simulated
    anti-paths: boolean
        use of antitethic variates
    mo_math:boolean
        use of moment matching
        '''
    
    if anti_paths is True:
        sn = npr.standard_normal((M+1,int(I/2)))
        sn = np.concatenate((sn,-sn),axis=1)
    else:
        sn = npr.standard_normal(( M+1 , I))
    if mo_match is True:
        sn = (sn-sn.mean())/sn.std()
    return sn

In [3]:
#European Option
S0= 100
r = 0.05
sigma = 0.25
T =1.0
I = 50000

In [11]:
def gbm_mcs_stat(K):
    '''Valuation o european call option using BlackScholesMerton model by montecarlo simulaion
    k:Strike Pirceof the option
    C0: pRESENT VALUE OF THE OPTIOn'''
    sn = gen_sn(1,I)
    #simulate index level at maturity
    ST = S0 * np.exp((r - 0.5*sigma**2)*T+ sigma*math.sqrt(T)*sn[1])
    #calculate the pay off maturity 
    hT = np.maximum(ST-K,0)
    #calculate MCS estimator
    C0 = math.exp(-r*T)*np.mean(hT)
    return ST,C0,hT

In [12]:
gbm_mcs_stat(K=110)

(array([ 88.93554605,  84.08815421,  91.86132199, ...,  85.36063223,
         83.6912982 , 133.97065615]),
 8.061873025067769,
 array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 23.97065615]))

In [30]:
#A more dynamic solutions for european call and  put option
M = 50
def gbm_mcs_call_put(K,option='call'):
    dt=T/M
    S= np.zeros((M+1,I))
    S[0] = 50
    sn = gen_sn(M,I)
    for t in range(1,M+1):
        S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt + sigma*math.sqrt(dt)*sn[t])
        if option =='call':
            hT = np.maximum(S[-1]-K,0)
        else:
            hT= np.maximum(K-S[-1],0)
            #discounting Fv rates
    CO = math.exp(-r*T)*np.mean(hT)
    return S[-1],CO

In [31]:
gbm_mcs_call_put(K=40,option='call')

(array([77.50646401, 64.87290553, 32.21561758, ..., 71.87530178,
        48.01159088, 66.39529052]),
 12.733659502550518)

In [32]:
gbm_mcs_call_put(K=40,option='put')

(array([60.10196343, 38.16755684, 46.00476058, ..., 64.49054069,
        56.0760832 , 60.22676625]),
 0.7600613747101098)